## Lab 1: Creating an American Red Cross Customer Service Chatbot

### Overview

[Amazon Bedrock AgentCore](https://aws.amazon.com/bedrock/agentcore/) helps you deploying and operating AI agents securely at scale - using any framework and model. It provides you with the capability to move from prototype to production faster. 

In this 6-labs tutorial, we will demonstrate the end-to-end journey from prototype to production using an **American Red Cross Customer Service Chatbot**. For this example we will use [Strands Agents](https://strandsagents.com/latest/), a simple-to-use, code-first framework for building agents and the Anthropic Claude Haiku 4.5 model from Amazon Bedrock. For your application you can use the framework and model of your choice. It's important to note that the concepts covered here can be applied using other frameworks and models as well.

**Workshop Journey:**
- **Lab 1 (Current)**: Create Agent Prototype - Build a functional chatbot with access to multiple knowledge bases
- **Lab 2**: Enhance with Memory - Add conversation context and personalization
- **Lab 3**: Scale with Gateway & Identity - Share tools across agents securely
- **Lab 4**: Deploy to Production - Use AgentCore Runtime with observability
- **Lab 5**: Evaluate Agent Performance - Monitor quality with online evaluations
- **Lab 6**: Build User Interface - Create a customer-facing application

In this first lab, we'll build an American Red Cross Customer Service Chatbot prototype that will evolve throughout the workshop into a production-ready system serving multiple users with persistent memory, shared tools, and full observability. Our chatbot will have access to three separate knowledge bases covering different lines of business:
- **search_biomedical_knowledge_base()** - Access Biomedical Services information (blood drives, appointments, locations)
- **search_humanitarian_knowledge_base()** - Access Humanitarian Services information (relief centers, grant applications)
- **search_training_services_knowledge_base()** - Access Training Services information (first aid classes, schedules, registrations)
- **web_search()** - Search the web for general information


### Architecture for Lab 1
<div style="text-align:left">
    <img src="images/architecture_lab1_strands.png" width="75%"/>
</div>

*Simple prototype running locally. In subsequent labs, we'll migrate this to AgentCore services with shared tools, persistent memory, and production-grade observability.*

### Prerequisites

* **AWS Account** with appropriate permissions
* **Python 3.10+** installed locally
* **AWS CLI configured** with credentials
* **Anthropic Claude 3.7** enabled on [Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)
* **Strands Agents** and other libraries installed in the next cells

#### Not using an AWS workshop account? 

If you are running this as a self-paced lab you MUST do additional 2 steps to create and deploy the cloudformation stack:

**Step 0.1: [Only for Self-paced labs]** 

Uncomment the cell below with command `!aws sts get-caller-identity` to know your Sagemaker Role. Proceed to IAM in the AWS console and search for your Sagemaker Role. Now add the [IAM Policy, AWS managed policies and Trust relationships](https://catalog.us-east-1.prod.workshops.aws/workshops/850fcd5c-fd1f-48d7-932c-ad9babede979/en-US/00-prerequisites/02-self-paced#iam-policy-for-bedrock-agentcore-workshop) as described in the [workshop self-paced prerequisites](https://catalog.us-east-1.prod.workshops.aws/workshops/850fcd5c-fd1f-48d7-932c-ad9babede979/en-US/00-prerequisites/02-self-paced) lab. 

In [ ]:
# Note: Uncomment and run only for self-paced labs
# !aws sts get-caller-identity 

**Step 0.2: [Only for Self-paced labs]**

Now, once you have all the required permissions for this `prereq.sh` script, run the following command to deploy the cloudformation template.

In [ ]:
# Note: Uncomment and run only for self-paced labs
# !bash scripts/prereq.sh

### Step 1: Install Dependencies and Import Libraries
Before we start, let's install the dependencies for this lab. You will see some dependency errors - they're safe to ignore for the scope of this workshop.

In [21]:
# Install required packages 
%pip install -U -r requirements.txt -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.4.0 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
sagemaker-studio 1.1.4 requires pydynamodb>=0.7.4, which is not installed.
aiobotocore 2.22.0 requires botocore<1.37.4,>=1.37.2, but you have botocore 1.42.30 which is incompatible.
autogluon-multimodal 1.4.0 requires transformers[sentencepiece]<4.50,>=4.38.0, but you have transformers 4.57.3 which is incompatible.
autogluon-timeseries 1.4.0 requires transformers[sentencepiece]<4.50,>=4.38.0, but you have transformers 4.57.3 which is incompatible.
sagemaker-studio-analytics-extension 0.2.2 requires sparkmagic==0.22.0, but you have sparkmagic 0.21.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


We can now import the required libraries and initialize our boto3 session

In [22]:
# Import libraries
import boto3
from boto3.session import Session

from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

from strands.tools import tool

In [23]:
# Get boto session
boto_session = Session()
region = boto_session.region_name

### Step 2: Implementing Knowledge Base Tools

Next, we will implement the knowledge base tools which will be provided to the American Red Cross Customer Service Chatbot. These tools allow the chatbot to search three separate knowledge bases for different lines of business.

Defining tools in Strands Agent is extremely simple, just add a `@tool` decorator to your function, and provide a description of the tool in the function's docstring. Strands Agents will use the function documentation, typing and arguments to provide context on this tool to your agent.

**Note:** Before running this lab, make sure you have:
1. Deployed the CloudFormation infrastructure stack
2. The CloudFormation stack automatically creates three knowledge bases (Biomedical, Humanitarian, Training Services) and stores their IDs in Parameter Store.
3. If you recently deployed, wait 5-10 minutes for the KnowledgeBaseSetup Custom Resource to finish. 


#### Tool 1: Search Biomedical Services Knowledge Base

**Purpose:** This tool allows the chatbot to search the Biomedical Services knowledge base for information about blood drives, appointments, locations, and sign-up information. Users can query about existing appointments, upcoming blood drives, eligibility requirements, and how to schedule donations.

In [ ]:
from strands_tools import retrieve
import boto3

@tool
def search_biomedical_knowledge_base(query: str) -> str:
    """
    Search the Biomedical Services knowledge base for information about blood drives, appointments, locations, and sign-up.

    Args:
        query: The search query about blood drives, appointments, locations, or sign-up information

    Returns:
        Relevant information from the Biomedical Services knowledge base
    """
    try:
        # Get KB ID from parameter store
        ssm = boto3.client("ssm")
        account_id = boto3.client("sts").get_caller_identity()["Account"]
        region = boto3.Session().region_name

        kb_id = ssm.get_parameter(Name=f"/{account_id}-{region}/kb/biomedical/knowledge-base-id")[
            "Parameter"
        ]["Value"]
        print(f"Successfully retrieved Biomedical KB ID: {kb_id}")

        # Use strands retrieve tool
        tool_use = {
            "toolUseId": "biomedical_query",
            "input": {
                "text": query,
                "knowledgeBaseId": kb_id,
                "region": region,
                "numberOfResults": 3,
                "score": 0.4,
            },
        }

        result = retrieve.retrieve(tool_use)

        if result["status"] == "success":
            return result["content"][0]["text"]
        else:
            return f"Unable to access Biomedical Services knowledge base. Error: {result['content'][0]['text']}"

    except Exception as e:
        print(f"Detailed error in search_biomedical_knowledge_base: {str(e)}")
        return f"Unable to access Biomedical Services knowledge base. Error: {str(e)}"


print("✅ Biomedical knowledge base tool ready")

✅ Return policy tool ready


#### Tool 2: Search Humanitarian Services Knowledge Base

**Purpose:** This tool allows the chatbot to search the Humanitarian Services knowledge base for information about relief centers, grant applications, aid programs, and application status. Users can find relief center locations, learn how to apply for grants, and check the status of their applications.

In [ ]:
@tool
def search_humanitarian_knowledge_base(query: str) -> str:
    """
    Search the Humanitarian Services knowledge base for information about relief centers, grant applications, and aid programs.

    Args:
        query: The search query about relief centers, grant applications, aid programs, or application status

    Returns:
        Relevant information from the Humanitarian Services knowledge base
    """
    try:
        # Get KB ID from parameter store
        ssm = boto3.client("ssm")
        account_id = boto3.client("sts").get_caller_identity()["Account"]
        region = boto3.Session().region_name

        kb_id = ssm.get_parameter(Name=f"/{account_id}-{region}/kb/humanitarian/knowledge-base-id")[
            "Parameter"
        ]["Value"]
        print(f"Successfully retrieved Humanitarian KB ID: {kb_id}")

        # Use strands retrieve tool
        tool_use = {
            "toolUseId": "humanitarian_query",
            "input": {
                "text": query,
                "knowledgeBaseId": kb_id,
                "region": region,
                "numberOfResults": 3,
                "score": 0.4,
            },
        }

        result = retrieve.retrieve(tool_use)

        if result["status"] == "success":
            return result["content"][0]["text"]
        else:
            return f"Unable to access Humanitarian Services knowledge base. Error: {result['content'][0]['text']}"

    except Exception as e:
        print(f"Detailed error in search_humanitarian_knowledge_base: {str(e)}")
        return f"Unable to access Humanitarian Services knowledge base. Error: {str(e)}"


print("✅ Humanitarian knowledge base tool ready")

✅ get_product_info tool ready


#### Tool 3: Search Training Services Knowledge Base

**Purpose:** This tool allows the chatbot to search the Training Services knowledge base for information about first aid training classes, schedules, locations, sign-up, and account information. Users can find available classes, check their registrations, and get information about their training accounts.

In [ ]:
@tool
def search_training_services_knowledge_base(query: str) -> str:
    """
    Search the Training Services knowledge base for information about first aid training classes, schedules, locations, sign-up, and account information.

    Args:
        query: The search query about training classes, schedules, locations, sign-up, or account information

    Returns:
        Relevant information from the Training Services knowledge base
    """
    try:
        # Get KB ID from parameter store
        ssm = boto3.client("ssm")
        account_id = boto3.client("sts").get_caller_identity()["Account"]
        region = boto3.Session().region_name

        kb_id = ssm.get_parameter(Name=f"/{account_id}-{region}/kb/training/knowledge-base-id")[
            "Parameter"
        ]["Value"]
        print(f"Successfully retrieved Training Services KB ID: {kb_id}")

        # Use strands retrieve tool
        tool_use = {
            "toolUseId": "training_query",
            "input": {
                "text": query,
                "knowledgeBaseId": kb_id,
                "region": region,
                "numberOfResults": 3,
                "score": 0.4,
            },
        }

        result = retrieve.retrieve(tool_use)

        if result["status"] == "success":
            return result["content"][0]["text"]
        else:
            return f"Unable to access Training Services knowledge base. Error: {result['content'][0]['text']}"

    except Exception as e:
        print(f"Detailed error in search_training_services_knowledge_base: {str(e)}")
        return f"Unable to access Training Services knowledge base. Error: {str(e)}"


print("✅ Training Services knowledge base tool ready")

✅ Web search tool ready


#### Tool 4: Web Search

**Purpose:** This tool allows the chatbot to search the web for general information, current events, or additional resources that may not be in the knowledge bases.


In [ ]:
@tool
def web_search(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """Search the web for updated information.

    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.

    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "Rate limit reached. Please try again later."
    except DDGSException as e:
        return f"Search error: {e}"
    except Exception as e:
        return f"Search error: {str(e)}"


print("✅ Web search tool ready")


### Step 3: Knowledge Base Setup

**Note:** The knowledge bases are created automatically when you deploy the CloudFormation stack (`prereq.ps1` or `prereq.sh`). Run the cells below in order:

1. **Verify** - Check that the three KBs are configured (SSM parameters). If you see the "previous single knowledge base" message, run `prereq.ps1` (or `prereq.sh`) to update the stack.
2. **Download** - Download knowledge base files from S3 to local `knowledge_base_data/` (CloudFormation already uploaded them to the kb-data-bucket).
3. **Sync** - Run ingestion jobs for each of the three knowledge bases to load the data into the vector stores.

Files in S3 use prefixes: `biomedical/`, `humanitarian/`, `training/`.

In [ ]:
# Verify knowledge bases are set up
import boto3

ssm = boto3.client("ssm")
account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name

try:
    biomedical_kb = ssm.get_parameter(Name=f"/{account_id}-{region}/kb/biomedical/knowledge-base-id")
    humanitarian_kb = ssm.get_parameter(Name=f"/{account_id}-{region}/kb/humanitarian/knowledge-base-id")
    training_kb = ssm.get_parameter(Name=f"/{account_id}-{region}/kb/training/knowledge-base-id")
    
    print("✅ All three knowledge bases are configured:")
    print(f"   - Biomedical Services KB ID: {biomedical_kb['Parameter']['Value']}")
    print(f"   - Humanitarian Services KB ID: {humanitarian_kb['Parameter']['Value']}")
    print(f"   - Training Services KB ID: {training_kb['Parameter']['Value']}")
except Exception as e:
    try:
        old_kb = ssm.get_parameter(Name=f"/{account_id}-{region}/kb/knowledge-base-id")
        print("⚠️  Detected the previous single knowledge base setup.")
        print("   Please update your CloudFormation stack to the latest template:")
        print("   Run: bash scripts/prereq.sh  (or prereq.ps1 on Windows)")
        print("   This will create the three Red Cross knowledge bases (Biomedical, Humanitarian, Training).")
    except Exception:
        print("⚠️  Knowledge bases not found. Please ensure the CloudFormation stack has been deployed.")
        print("   Run: bash scripts/prereq.sh  (or prereq.ps1 on Windows)")
        print("   If you recently deployed/updated, wait 5-10 minutes for the KnowledgeBaseSetup to finish.")
        print(f"   Error: {str(e)}")


Downloaded: laptop-maintenance-guide.txt
Downloaded: monitor-calibration-guide.txt
Downloaded: smartphone-setup-guide.txt
Downloaded: troubleshooting-guide.txt
Downloaded: warranty-service-guide.txt
Downloaded: wireless-connectivity-guide.txt
All files saved to: knowledge_base_data/


In [ ]:
import os
import boto3

def download_files():
    account_id = boto3.client("sts").get_caller_identity()["Account"]
    region = boto3.Session().region_name
    bucket_name = f"{account_id}-{region}-kb-data-bucket"

    os.makedirs("knowledge_base_data", exist_ok=True)

    s3 = boto3.client("s3")
    objects = s3.list_objects_v2(Bucket=bucket_name)

    for obj in objects.get("Contents", []):
        file_name = obj["Key"]
        s3.download_file(bucket_name, file_name, f"knowledge_base_data/{file_name}")
        print(f"Downloaded: {file_name}")

    print("All files saved to: knowledge_base_data/")

download_files()


#### (Optional) Sync Knowledge Bases

**Note:** After CloudFormation creates the knowledge bases and data sources, you may need to trigger ingestion jobs to sync the data. You typically don't need to run this cell unless:
- The initial ingestion jobs didn't complete successfully
- You've updated the knowledge base files in S3 after initial setup
- You want to manually trigger a sync

To check ingestion status, go to the AWS Bedrock console → Knowledge Bases → select a KB → Data Sources → Ingestion Jobs.

In [ ]:
import time
import boto3

ssm = boto3.client("ssm")
bedrock = boto3.client("bedrock-agent")
s3 = boto3.client("s3")

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name
bucket_name = f"{account_id}-{region}-kb-data-bucket"

lobs = ["biomedical", "humanitarian", "training"]

for lob in lobs:
    kb_id = ssm.get_parameter(Name=f"/{account_id}-{region}/kb/{lob}/knowledge-base-id")["Parameter"]["Value"]
    ds_id = ssm.get_parameter(Name=f"/{account_id}-{region}/kb/{lob}/data-source-id")["Parameter"]["Value"]

    s3_objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=f"{lob}/")
    file_names = [obj["Key"] for obj in s3_objects.get("Contents", [])]

    response = bedrock.start_ingestion_job(
        knowledgeBaseId=kb_id, dataSourceId=ds_id, description=f"Quick sync {lob}"
    )
    job_id = response["ingestionJob"]["ingestionJobId"]
    print(f"Bedrock knowledge base sync job started for {lob}, ingesting the data files from s3")

    while True:
        job = bedrock.get_ingestion_job(
            knowledgeBaseId=kb_id, dataSourceId=ds_id, ingestionJobId=job_id
        )["ingestionJob"]
        status = job["status"]
        if status in ["COMPLETE", "FAILED"]:
            break
        time.sleep(10)

    if status == "COMPLETE":
        file_count = job.get("statistics", {}).get("numberOfDocumentsScanned", 0)
        files_list = ", ".join(file_names)
        print(f"Bedrock knowledge base sync job completed Successfully for {lob}, ingested {file_count} files")
        print(f"Files ingested: {files_list}")
    else:
        print(f"Bedrock knowledge base sync job failed for {lob} with status: {status}")


Bedrock knowledge base sync job started, ingesting the data files from s3
Bedrock knowledge base sync job completed Successfully, ingested 6 files
Files ingested: laptop-maintenance-guide.txt, monitor-calibration-guide.txt, smartphone-setup-guide.txt, troubleshooting-guide.txt, warranty-service-guide.txt, wireless-connectivity-guide.txt


### Step 4: Create and Configure the American Red Cross Chatbot

Now we'll create the chatbot with all the knowledge base tools and a system prompt that reflects the Red Cross mission and services.

In [ ]:
from strands.models import BedrockModel
from strands import Agent

SYSTEM_PROMPT = """You are a helpful and professional customer service chatbot for the American Red Cross.
Your role is to:
- Provide accurate information using the knowledge bases and tools available to you
- Assist users with questions about Biomedical services (blood drives, appointments, locations)
- Help with Humanitarian services (relief centers, grant applications, aid programs)
- Support Training Services inquiries (first aid classes, schedules, registrations)
- Be friendly, patient, and understanding with all users
- Always offer additional help after answering questions
- If you can't help with something, direct users to the appropriate Red Cross contact or resource

You have access to the following tools:
1. search_biomedical_knowledge_base() - For blood drive appointments, locations, and sign-up information
2. search_humanitarian_knowledge_base() - For relief center locations, grant applications, and aid programs
3. search_training_services_knowledge_base() - For first aid training classes, schedules, locations, and account information
4. web_search() - To search the web for current information or additional resources

Always use the appropriate knowledge base tool to get accurate, up-to-date information. Use web_search for general information or when knowledge bases don't contain the specific information needed."""

# Initialize the Bedrock model (Anthropic Claude Haiku 4.5)
model = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    temperature=0.3,
    region_name=region,
)

# Create the American Red Cross chatbot with all knowledge base tools
agent = Agent(
    model=model,
    tools=[
        search_biomedical_knowledge_base,  # Tool 1: Biomedical Services KB
        search_humanitarian_knowledge_base,  # Tool 2: Humanitarian Services KB
        search_training_services_knowledge_base,  # Tool 3: Training Services KB
        web_search,  # Tool 4: Web search for general information
    ],
    system_prompt=SYSTEM_PROMPT,
)

print("✅ American Red Cross Customer Service Chatbot created successfully!")

✅ Technical support tool ready


### Step 5: Test the American Red Cross Chatbot

Let's test our chatbot with sample queries to ensure all knowledge base tools work correctly.

In [30]:
SYSTEM_PROMPT = """You are a helpful and professional customer support assistant for an electronics e-commerce company.
Your role is to:
- Provide accurate information using the tools available to you
- Support the customer with technical information and product specifications, and maintenance questions
- Be friendly, patient, and understanding with customers
- Always offer additional help after answering questions
- If you can't help with something, direct customers to the appropriate contact

You have access to the following tools:
1. get_return_policy() - For warranty and return policy questions
2. get_product_info() - To get information about a specific product
3. web_search() - To access current technical documentation, or for updated information. 
4. get_technical_support() - For troubleshooting issues, setup guides, maintenance tips, and detailed technical assistance
For any technical problems, setup questions, or maintenance concerns, always use the get_technical_support() tool as it contains our comprehensive technical documentation and step-by-step guides.

Always use the appropriate tool to get accurate, up-to-date information rather than making assumptions about electronic products or specifications."""

# Initialize the Bedrock model (Anthropic Claude 3.7 Sonnet)
model = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    temperature=0.3,
    region_name=region,
)

# Create the customer support agent with all tools
agent = Agent(
    model=model,
    tools=[
        get_product_info,  # Tool 1: Simple product information lookup
        get_return_policy,  # Tool 2: Simple return policy lookup
        web_search,  # Tool 3: Access the web for updated information
        get_technical_support,  # Tool 4: Technical support & troubleshooting
    ],
    system_prompt=SYSTEM_PROMPT,
)

print("Customer Support Agent created successfully!")

Customer Support Agent created successfully!


### Step 5: Test the Customer Support Agent

Let's test our agent with sample queries to ensure all tools work correctly.

#### Test Return check

In [31]:
response = agent("What's the return policy for my thinkpad X1 Carbon?")

I'll get the return policy information for laptops for you.
Tool #1: get_return_policy
Great! Here's the return policy for your ThinkPad X1 Carbon laptop:

**Return Policy for Laptops:**

- **Return Window:** 30 days from delivery
- **Condition Requirements:** Must be in original packaging with all accessories included and no software modifications
- **Process:** Technical support verification is required before initiating a return
- **Refund Timeline:** 7-10 business days after inspection
- **Shipping:** Free return shipping is provided if you have the original packaging
- **Warranty:** Your laptop includes a 1-year manufacturer warranty, and extended warranty options are available

**Important notes:**
- Make sure your laptop is in its original condition with all original accessories
- You'll need to go through technical support verification before starting the return process
- Keep your original packaging to qualify for free return shipping

Is there anything else you'd like to know

In [32]:
response = agent("My laptop won't turn on, what should I check?")

I'll get you some troubleshooting steps for your laptop not turning on.
Tool #2: get_technical_support
Successfully retrieved KB ID: EYHEAWHTGA
Here are the troubleshooting steps for your laptop that won't turn on:

**Basic Power Troubleshooting:**

1. **Check Power Cable Connections**
   - Make sure the power cable is firmly connected to both the laptop and the power outlet
   - Look for any visible damage to the cable

2. **Verify Power Outlet Functionality**
   - Try plugging the power cable into a different outlet
   - Test the outlet with another device (like a lamp) to confirm it's working

3. **Try a Different Power Cable**
   - If available, test with an alternative power cable to rule out cable failure

4. **Check for Physical Damage**
   - Inspect the laptop for any signs of physical damage
   - Look at the charging port for any debris or damage

**Next Steps:**

If you've tried all of these steps and your laptop still won't turn on, here are some additional things to conside

#### Test 3: Training Services Query

In [ ]:
# Test query about training classes
response = agent("What first aid classes are available in April? I'm looking for classes in Seattle.")
print(response.message["content"][0]["text"])


In [33]:
response = agent(
    "I bought an iphone 14 last month. I don't like it because it heats up. How do I solve it?"
)

I'll get you some troubleshooting steps for the overheating issue with your iPhone 14.
Tool #3: get_technical_support
Successfully retrieved KB ID: EYHEAWHTGA
I found some battery optimization information that can help with overheating. Let me also check the return policy for smartphones since you mentioned you don't like the phone.
Tool #4: get_return_policy
Great! Here are some solutions for your iPhone 14 overheating issue:

**Battery & Performance Optimization Steps:**

1. **Enable Battery Optimization Settings**
   - Go to Settings > Battery and enable Battery Saver Mode
   - This reduces performance but helps manage heat

2. **Adjust Screen Brightness and Timeout**
   - Lower screen brightness (high brightness generates heat)
   - Reduce screen timeout duration
   - Disable auto-brightness if it's constantly adjusting

3. **Manage Background App Refresh**
   - Go to Settings > General > Background App Refresh
   - Disable for apps that don't need it
   - Background activity gener

## 🎉 Lab 1 Complete!

You've successfully created a functional Customer Support Agent prototype! Here's what you accomplished:

- Built an agent with 3 custom tools (return policy, product info, web search)  
- Tested multi-tool interactions and web search capabilities  
- Established the foundation for our production journey  

### Current Limitations (We'll fix these!)
- **Single user conversation memory** - local conversation session, multiple customers need multiple sessions.
- **Conversation history limited to session** - no long term memory or cross session information is available in the conversation.
- **Tools reusability** - tools aren't reusable across different agents  
- **Running locally only** - not scalable
- **Identity** - No user and/or agent identity or access control
- **Observability** - Limited observability into agent behavior
- **Existing APIs** - No access to existing enterprise APIs for customer data

##### Next Up [Lab 2: Personalize our agent by adding memory →](lab-02-agentcore-memory.ipynb)
